In [ ]:
import sys
import os

notebook_dir        = os.getcwd()
data_grabber_path   = os.path.abspath(os.path.join(notebook_dir, '../data_grabbers'))
sys.path.append(data_grabber_path)

In [ ]:
from api.coingecko import Main
from request_models.coingecko.main import CoinDataByID
import copy
from abc import ABC, abstractmethod
import pandas as pd
from tabulate import tabulate

In [ ]:
class Portfolio:
    def __init__(self):
        self.api = Main()
        _holding = {'usd_value': 0.0, 'meta': None}
        self.holdings = {
            'bitcoin': {**_holding, 'token_qty': 0.01},
            'solana': {**_holding, 'token_qty': 0.01},
            'tether': {**_holding, 'token_qty': 0.01}
        }

    async def update_holdings(self):
        for token_id in self.holdings.keys():
            res = await self.api.request(f'coins/{token_id}', 'GET', headers=self.api.headers)
            validated_meta = CoinDataByID.Validator(**res)
            self.holdings[token_id]['meta'] = validated_meta
            self.holdings[token_id]['usd_value'] = self.holdings[token_id]['token_qty'] * validated_meta.market_data.current_price.usd

    def get_portfolio_usd_sum(self, holdings=None):
        _holdings = self.holdings if holdings is None else holdings
        return sum([v['usd_value'] for v in _holdings.values()])

    def print_holdings_and_pct(self, comaparitive_holdings: dict=None):
        p = self.construct_holdings_dataframe(self.holdings)
        if comaparitive_holdings is not None:
            p2 = self.construct_holdings_dataframe(comaparitive_holdings)
            p['token_qty_delta']    = (p2['token_qty'] - p['token_qty']).map('{:,.6f}'.format)
            p['usd_value_delta']    = (p2['usd_value'] - p['usd_value']).map('${:,.2f}'.format)
            p['new_pct_allocation'] = p2['pct. allocation'].map('{:.2%}'.format)

        p['usd_value']              = p['usd_value'].map('${:,.2f}'.format)
        p['pct. allocation']        = p['pct. allocation'].map('{:.2%}'.format)

        print(tabulate(p, headers='keys', tablefmt='grid'))

    @staticmethod
    def construct_holdings_dataframe(holdings: dict):
        data                  = {token: {key: value for key, value in details.items() if key != 'meta'} for token, details in holdings.items()}
        df                    = pd.DataFrame.from_dict(data, orient='index')
        df['pct. allocation'] = df['usd_value']/df['usd_value'].sum()
        return df

    @staticmethod
    def categorize_token(mcap_usd):
        if mcap_usd < 50_000_000: return 'microcap'
        elif 50_000_000 <= mcap_usd < 300_000_000: return 'smallcap'
        elif 300_000_000 <= mcap_usd < 2_000_000_000: return 'midcap'
        elif mcap_usd >= 2_000_000_000: return 'largecap'

class BasePortfolioOptimizer(Portfolio, ABC):
    def __init__(self, existing_portfolio=None):
        super().__init__()
        if existing_portfolio:
            self.holdings = copy.deepcopy(existing_portfolio.holdings)

    @abstractmethod
    def optimize(self):
        """Implement optimization logic in subclasses."""
        pass

In [ ]:
class ReturnsWeightedOptimizer(BasePortfolioOptimizer):
    def __init__(self, existing_portfolio=None):
        super().__init__(existing_portfolio)
    
    # Categorize tokens and compute total value for each category
    def bucket_assets(self):
        categorization = {}
        for token, details in self.holdings.items():
            if token == 'tether': continue
            category = self.categorize_token(details['meta'].market_data.market_cap.usd)
            bucket = 'midcap_largecap' if category in ['midcap', 'largecap'] else 'microcap_smallcap'
            categorization[token] = bucket

    def optimize(self):
        total_portfolio_value = self.get_portfolio_usd_sum()
        category_cap = {'midcap_largecap': 0.5, 'microcap_smallcap': 0.3}
        category_limits = {'midcap_largecap': 0.15, 'microcap_smallcap': 0.05}
        price_changes = {token: details['meta'].market_data.price_change_percentage_60d_in_currency.usd
                         for token, details in self.holdings.items() if token != 'tether'}
        btc_price_change = self.holdings['bitcoin']['meta'].market_data.price_change_percentage_60d_in_currency.usd

        # Categorize tokens and compute total value for each category
        categorization = {}
        for token, details in self.holdings.items():
            if token == 'tether': continue
            category = self.categorize_token(details['meta'].market_data.market_cap.usd)
            bucket = 'midcap_largecap' if category in ['midcap', 'largecap'] else 'microcap_smallcap'
            categorization[token] = bucket

        # Process each holding based on categorization and rules
        for token, details in self.holdings.items():
            if token == 'tether':
                continue
            bucket = categorization[token]
            token_value = details['usd_value']
            token_pct_of_portfolio = token_value / total_portfolio_value
            max_pct_allowed = category_limits[bucket]

            # Check if the asset needs to be reduced to its max percentage
            if token_pct_of_portfolio > max_pct_allowed:
                excess_value = token_value - (max_pct_allowed * total_portfolio_value)
                print(f"SELL TOKEN ({token}): Max % Breached. token_value: {token_value} | max_pct_allowed: {max_pct_allowed} | total_portfolio_value: {total_portfolio_value}")
                self.sell_token(token, excess_value)

            # Check performance against BTC and mark for selling if underperforming
            if bucket in ['midcap_largecap'] and price_changes[token] < btc_price_change:
                print(f"SELL TOKEN ({token}):Underperforming BTC!. price_changes[token]: {price_changes[token]} | btc_price_change: {btc_price_change}")
                self.sell_token(token, token_value)  # Selling strategy may vary based on specific needs

        # Categorize tokens and compute total value for each category
        bucket_values = {'midcap_largecap': 0, 'microcap_smallcap': 0}
        for token, details in self.holdings.items():
            if token == 'tether': continue
            category = self.categorize_token(details['meta'].market_data.market_cap.usd)
            bucket = 'midcap_largecap' if category in ['midcap', 'largecap'] else 'microcap_smallcap'
            bucket_values[bucket] += details['usd_value']

        # Reduce category values if they exceed total allowed percentage
        for bucket, total_value in bucket_values.items():
            if total_value > (category_cap[bucket] * total_portfolio_value):
                print(f"REDUCE BUCKET ({bucket}): total_value: {total_value} | allowable_amount: {category_cap[bucket] * total_portfolio_value}")
                excess_value = total_value - (category_cap[bucket] * total_portfolio_value)
                self.reduce_bucket(bucket, excess_value, price_changes, categorization)

    def sell_token(self, token, amount):
        current_price = self.holdings[token]['meta'].market_data.current_price.usd
        amount_to_reduce_tokens = amount / current_price
        self.holdings[token]['token_qty'] -= amount_to_reduce_tokens
        self.holdings[token]['usd_value'] -= amount
        self.holdings['tether']['usd_value'] += amount

    def reduce_bucket(self, bucket, excess_value, price_changes, categorization):
        # Determine reduction based on the worst performance in 60 days
        eligible_tokens = [token for token, cat in categorization.items() if cat == bucket]
        total_performance_weight = sum(price_changes[token] for token in eligible_tokens)

        # Distribute the excess_value based on performance weights
        for token in eligible_tokens:
            token_value = self.holdings[token]['usd_value']
            # Calculate the proportion of the token's performance to the total performance weight
            performance_weight = price_changes[token] / total_performance_weight
            reduction_amount = excess_value * performance_weight

            # Ensure we do not sell more than what the token's current value is
            reduction_amount = min(reduction_amount, token_value)
            self.sell_token(token, reduction_amount)


In [ ]:
portfolio = Portfolio()
await portfolio.update_holdings()
optimizer = ReturnsWeightedOptimizer(portfolio)

In [ ]:
portfolio.print_holdings_and_pct()

In [ ]:
optimizer.optimize()
portfolio.print_holdings_and_pct(optimizer.holdings)

In [ ]:
portfolio.print_holdings_and_pct()

In [ ]:
portfolio.get_portfolio_usd_sum()